In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from googleapiclient.discovery import build
import pandas as pd
import json
from collections import OrderedDict

In [4]:
df = pd.read_excel('/content/drive/MyDrive/KOBIS_역대_박스오피스_내역(공식통계_기준)_2022-06-26.xlsx', 
                   usecols = ['영화명', '개봉일', '매출액', 'Unnamed: 5'], header = 4)
df.head()

,영화명,개봉일,매출액,Unnamed: 5
0,명량,2014-07-30,1.357484e+11,17613682.0
1,극한직업,2019-01-23,1.396480e+11,16264944.0
2,신과함께-죄와 벌,2017-12-20,1.156987e+11,14410754.0
3,국제시장,2014-12-17,1.109135e+11,14257115.0
4,어벤져스: 엔드게임,2019-04-24,1.221827e+11,13934592.0


In [5]:
df.rename(columns={'Unnamed: 5':'관객수(S: 서울 기준)'}, inplace=True)
df.head()

,영화명,개봉일,매출액,관객수(S: 서울 기준)
0,명량,2014-07-30,1.357484e+11,17613682.0
1,극한직업,2019-01-23,1.396480e+11,16264944.0
2,신과함께-죄와 벌,2017-12-20,1.156987e+11,14410754.0
3,국제시장,2014-12-17,1.109135e+11,14257115.0
4,어벤져스: 엔드게임,2019-04-24,1.221827e+11,13934592.0


In [6]:
df = df.fillna(df.mean())
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   영화명            500 non-null    object        
 1   개봉일            500 non-null    datetime64[ns]
 2   매출액            500 non-null    float64       
 3   관객수(S: 서울 기준)  500 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 15.8+ KB


In [7]:
df = df.head(50)
df

,영화명,개봉일,매출액,관객수(S: 서울 기준)
0,명량,2014-07-30,1.357484e+11,17613682.0
1,극한직업,2019-01-23,1.396480e+11,16264944.0
2,신과함께-죄와 벌,2017-12-20,1.156987e+11,14410754.0
3,국제시장,2014-12-17,1.109135e+11,14257115.0
4,어벤져스: 엔드게임,2019-04-24,1.221827e+11,13934592.0
5,겨울왕국 2,2019-11-21,1.148104e+11,13747792.0
6,아바타,2009-12-17,1.284471e+11,13624328.0
7,베테랑,2015-08-05,1.051682e+11,13414009.0
8,괴물,2006-07-27,0.000000e+00,13019740.0
9,도둑들,2012-07-25,9.366557e+10,12983330.0


In [8]:
df['개봉일'] = df['개봉일'].astype('str')

In [9]:
movie_name = df['영화명'].to_list()
opening_date = df['개봉일'].to_list()
sales_amount = df['매출액'].to_list()
number_of_spectoators = df['관객수(S: 서울 기준)'].to_list()

In [10]:
box_office_data = OrderedDict()

In [11]:
box_office_data['movie_name'] = movie_name
box_office_data['opening_date'] = opening_date
box_office_data['sales_amount'] = sales_amount
box_office_data['number_of_spectoators'] = number_of_spectoators

In [12]:
print(json.dumps(box_office_data, ensure_ascii=False, indent='\t'))

{
	"movie_name": [
		"명량 ",
		"극한직업 ",
		"신과함께-죄와 벌 ",
		"국제시장 ",
		"어벤져스: 엔드게임 ",
		"겨울왕국 2 ",
		"아바타 ",
		"베테랑 ",
		"괴물 ",
		"도둑들 ",
		"7번방의 선물 ",
		"암살 ",
		"알라딘 ",
		"광해, 왕이 된 남자 ",
		"왕의 남자 ",
		"신과함께-인과 연 ",
		"택시운전사 ",
		"태극기 휘날리며 ",
		"부산행 ",
		"해운대 ",
		"변호인 ",
		"어벤져스: 인피니티 워 ",
		"실미도 ",
		"어벤져스: 에이지 오브 울트론 ",
		"기생충 ",
		"겨울왕국 ",
		"인터스텔라 ",
		"보헤미안 랩소디 ",
		"검사외전 ",
		"엑시트 ",
		"설국열차 ",
		"관상 ",
		"아이언맨 3 ",
		"캡틴 아메리카: 시빌 워 ",
		"해적: 바다로 간 산적 ",
		"수상한 그녀 ",
		"국가대표 ",
		"디워 ",
		"백두산 ",
		"과속스캔들 ",
		"스파이더맨: 파 프롬 홈 ",
		"웰컴 투 동막골 ",
		"공조 ",
		"트랜스포머 3 ",
		"히말라야 ",
		"미션임파서블:고스트프로토콜 ",
		"스파이더맨: 노 웨이 홈 ",
		"트랜스포머: 패자의 역습 ",
		"밀정 ",
		"최종병기 활 "
	],
	"opening_date": [
		"2014-07-30",
		"2019-01-23",
		"2017-12-20",
		"2014-12-17",
		"2019-04-24",
		"2019-11-21",
		"2009-12-17",
		"2015-08-05",
		"2006-07-27",
		"2012-07-25",
		"2013-01-23",
		"2015-07-22",
		"2019-05-23",
		"2012-09-13",
		"2005-12-29",
		"2018-08-01",
		"2017-08-02",
		"2004-02-05",
		"2016-07-20",
		"

In [13]:
with open('box_office_etc.json', 'w', encoding='utf-8') as make_file:
  json.dump(box_office_data, make_file, ensure_ascii=False, indent='\t')